In [1]:
import pymysql

# Clase User
class User(object):
    def __init__(self):
        self.id = ""
        self.created = ""
        self.name = ""
        self.tag = ""
        self.location = ""
        self.description = None
        self.followersCount = 0
        self.friendsCount= 0
        self.favouritesCount = 0
        
    def __init__(self,id,fechaCreacion,nombre,tag,ubicacion,descripcion,followersCount,friendsCount,favouritesCount):
        self.id = id
        self.fechaCreacion = fechaCreacion
        self.nombre = nombre
        self.tag = tag
        self.ubicacion = ubicacion
        self.descripcion = descripcion
        self.followersCount = followersCount
        self.friendsCount = friendsCount
        self.favouritesCount = favouritesCount

    def getId(self):
        return self.id

    
    def agregarUsuario(self):
        try:
            connection = pymysql.connect(host = 'localhost', user = 'root',password = 'root',
                                         db = 'dbtwitter')
            
            with connection.cursor() as cursor:
                sentenciaSQL = "INSERT INTO dbtwitter.usuario(id,fechaCreacion,nombre,tag,ubicacion,descripcion,followersCount,friendsCount,favouritesCount) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"
                
                cursor.execute(sentenciaSQL, (str(self.id), self.fechaCreacion, str(self.nombre), str(self.tag), str(self.ubicacion), 
                                              str(self.descripcion), self.followersCount, self.friendsCount, self.favouritesCount))
                connection.commit()
                
        except Exception as e:
            print("Error on_data: %s" % str(e))
        finally:
            connection.close()
                         
    def __str__(self):
        return "{0}|{1}|{2}|{3}|{4}|{5}|{6}|{7}".format(self.id, 
                                                        self.fechaCreacion, 
                                                        self.nombre,
                                                        self.tag,
                                                        self.ubicacion,
                                                        self.descripcion,
                                                        self.followersCount,
                                                        self.friendsCount,
                                                        self.favouritesCount)

In [2]:
class Tweet(object):
    
    #metodo para inicializar la clase
    def __init__(self):
        self.id = ""
        self.fechaCreacion = ""
        self.contenido = ""
        self.user = None
   
    def __init__(self,id,fechaCreacion,contenido,user):
        self.id = id
        self.fechaCreacion = fechaCreacion
        self.contenido = contenido
        self.user = user
    
    def agregarTweet(self):
        try:
            connection = pymysql.connect(host = 'localhost', user = 'root',password = 'root',
                                         db = 'dbtwitter')
            
            with connection.cursor() as cursor:             
                sentenciaSQL=("INSERT INTO dbtwitter.tweet(id,fechaCreacion,texto,idUsuario) VALUES (%s,%s,%s,%s)")
                
                cursor.execute(sentenciaSQL, (str(self.id), self.fechaCreacion, str(self.contenido), str(self.user)))
                
                connection.commit()
                
        except Exception as e:
            print("Error on_data: %s" % str(e))
        finally:
            connection.close()
    
    def __str__(self):
        return "{0}|{1}|{2}|{3}".format(self.id,self.fechaCreacion,self.contenido,self.user)

In [3]:
import re
import emoji
import time
from datetime import datetime

class UtilTwitter(object):
    
    emojin_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F" # emoticons
                                u"\U0001F300-\U0001F5FF" # symbols & pictographs
                                u"\U0001F680-\U0001F6FF" # transport & map symbols
                                u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                                u"\U00002702-\U000027B0"
                                u"\U00002600-\U000027BF"
                                u"\U0001f300-\U0001f64F"
                                u"\U000024C2-\U0001F251"
                                "]+", flags=re.UNICODE)
    
    @staticmethod
    def format_datetime(text):
        datetime_object = datetime.strptime(text,'%a %b %d %H:%M:%S %z %Y')
        return datetime_object.strftime('%Y-%m-%d %H:%M:%S')
    
    @staticmethod
    def remove_emoticon(text):
        free_emoticon = ''.join(c for c in text if c not in emoji.UNICODE_EMOJI)
        return UtilTwitter.emojin_pattern.sub(r'', free_emoticon)
    
    @staticmethod
    def clear_text(text:str):
        if text is None:
            return "Indefinido"
        elif text.strip() == "":
            return "Indefinido"
        else:
            return " ".join(re.findall("[a-zA-Z]+", text))

In [4]:
import unittest

class UtilTwitterTest(unittest.TestCase):
    
    def test_format_datetime(self):
        self.assertEqual("2018-11-07 07:32:57",UtilTwitter.format_datetime('Wed Nov 07 07:32:57 +0000 2018'))
    
    def test_clear_text(self):
        self.assertEqual("nadya", UtilTwitter.clear_text("nadya🥀"))
                                                                           
    def test_clear_none(self):
        self.assertEqual("Indefinido", UtilTwitter.clear_text(None))
                                                                           
    def test_clear_vacio(self):
        self.assertEqual("Indefinido", UtilTwitter.clear_text("  "))
        ###Completar código###
                                                                           
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

....
----------------------------------------------------------------------
Ran 4 tests in 0.003s

OK


In [5]:
import tweepy

import time
import json

class TwitterListener(tweepy.StreamListener):
    """Custom StreamListener for streaming data."""
    def __init__(self, time_limit=60):
        self.start_time = time.time()
        self.limit = time_limit
        
    def on_data(self, tweet_json):
        if (time.time() - self.start_time) < self.limit:
            tweet_json = tweet_json.encode('utf-8')
            decoded = json.loads(tweet_json)
            print(decoded)
            try:
                #Obtener datos Usuario
                userjson = decoded.get('user')
                id_user = userjson.get('id')
                created_user = UtilTwitter.format_datetime(userjson.get('created_at'))
                name_user = UtilTwitter.clear_text(userjson.get('name'))
                tag_user = UtilTwitter.clear_text(userjson.get('screen_name'))
                location_user = UtilTwitter.clear_text(userjson.get('location'))
                description_user = UtilTwitter.clear_text(userjson.get('description'))
                followersCount_user = userjson.get('followers_count')
                friendsCount_user = userjson.get('friends_count')
                favouritesCount_user = userjson.get('favourites_count')

                #Obtener datos Tweet
                id_tweet = decoded.get('id_str')
                text_tweet = UtilTwitter.remove_emoticon(decoded.get('text'))
                created_tweet = UtilTwitter.format_datetime(decoded.get('created_at'))

                oUser = User(id_user,created_user,name_user,"@"+tag_user,location_user,description_user,followersCount_user,
                             friendsCount_user,favouritesCount_user)
                print(oUser)

                oTweet = Tweet(id_tweet, created_tweet, text_tweet, id_user)
                print(oTweet)

                oUser.agregarUsuario()
                oTweet.agregarTweet()

            except BaseException as e:
                print("Error on_data: %s" % str(e))
                time.sleep(5)
            return True
        else:
            return False #Kill current stream
        
    def on_error(self, status):
        print("Error status: " + str(status))
        return True
        
    def on_timeout(self):
        print ('Timeout...')
        return True # Don't kill the stream

In [6]:
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener

class ExtraccionTwitterCtrl(object):
    
    #Credenciales de cuenta Twitter
    CONSUMER_KEY = '9mwmv7Yt5R783Jry1bYEiGe2n'
    CONSUMER_SECRET = 'NQazRU4BpfmfwKmWD2pxhebAjitfEFOPHErOKTrWjBrn4jpXSE'
    ACCESS_TOKEN = '442102127-wvZfxbOkw3V8eQfbJgI7iSkyokI96tw8U2u5ROj0'
    ACCESS_SECRET = 'L0PJfiYiT0cToCgZKDv6fMJD64iPbr0V9D6S0eHiVXRuW'

    def __init__(self):
        # Autenticación
        self.auth = OAuthHandler(self.CONSUMER_KEY, self.CONSUMER_SECRET)
        self.auth.set_access_token(self.ACCESS_TOKEN, self.ACCESS_SECRET)
    
    def filterByText(self, list_tag, time_limit):
        print("Inicio de filterByText\n")
        try:
            twitter_stream = tweepy.streaming.Stream(self.auth, TwitterListener(time_limit))
            twitter_stream.filter(track=list_tag)
        except (KeyboardInterrupt, SystemExit):
            print("Parar proceso de extracción\n")
        print("Fin de filterByText\n")
     
    def filterByLocations(self, loc, time_limit):
        print("Inicio de filterByLocations\n")
        try:
            twitter_stream = tweepy.streaming.Stream(self.auth, TwitterListener(time_limit))
            twitter_stream.filter(locations = loc)
        except (KeyboardInterrupt, SystemExit):
            print("Parar proceso de extracción\n")
        print("Fin de filterByLocations\n")
        
    def filterByMe(self, list_tag, loc, time_limit):
        print("Inicio de filterByText\n")
        try:
            twitter_stream = tweepy.streaming.Stream(self.auth, TwitterListener(time_limit))
            twitter_stream.filter(track = list_tag, locations = loc, languages=['es'])
        except (KeyboardInterrupt, SystemExit):
            print("Parar proceso de extracción\n")
        print("Fin de filterByText\n")

In [7]:
hash_tag=['InnovaTech']

france=[-5.1406, 41.333740, 9.559320, 51.089062]
spain =[-4.042280, 40.611900, -4.023130,40.630329]
italy=[-96.901863,32.171291, -96.866524, 32.204861]
germany = [5.866240,47.270210, 15.042050,55.058140]
peru = [ -81.326736,-18.34972, -68.677979, -0.01297]
senegal =[ -17.535231,12.30727, -11.35588,16.691629]
costaMarfil =[-8.5993,4.35706, -2.49489,10.73664]

In [8]:
oExtraccion = ExtraccionTwitterCtrl()

In [9]:
# oExtraccion.filterByText(hash_tag, 100)

In [10]:
# oExtraccion.filterByLocations(peru, 10)

In [11]:
oExtraccion.filterByMe(hash_tag, peru, 100)

Inicio de filterByText

{'created_at': 'Thu Nov 08 00:53:45 +0000 2018', 'id': 1060334523903283201, 'id_str': '1060334523903283201', 'text': '@KeikoFujimori Tu misma estabas en contra del indulto .\nEstas recluida pq hay as de un indicio de todos los delitos… https://t.co/JskN4c1MVR', 'display_text_range': [15, 140], 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': True, 'in_reply_to_status_id': 1060291344244064256, 'in_reply_to_status_id_str': '1060291344244064256', 'in_reply_to_user_id': 53189543, 'in_reply_to_user_id_str': '53189543', 'in_reply_to_screen_name': 'KeikoFujimori', 'user': {'id': 169568386, 'id_str': '169568386', 'name': 'PASAJERO 101', 'screen_name': 'PASAJERO101', 'location': 'Panamá', 'url': None, 'description': 'Una persona no comun ni corriente. Tan diferente y amante de la vida.', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 246, 'friends_count': 147, 'listed_coun